# Installs and Imports

In [2]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import torch
from collections import Counter

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import torch
from collections import Counter

In [ ]:
corpus = pd.read_json('/content/Archive.json')

In [ ]:
len(corpus)

20000

# Geting keywords using Tfidf and Logistic Regression

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(2,3))

In [ ]:
text_corpus = corpus['TandA']
X = tfidf.fit_transform(text_corpus)
#feature_names = np.array(tfidf.get_feature_names_out())

In [ ]:
some = corpus['TandA']
responses = tfidf.transform(some)
feature_names = np.array(tfidf.get_feature_names_out())
def get_top_tf_idf_words(response, top_n=50):
    sorted_nzs = np.argsort(response.data)[:-(top_n+1):-1]
    return feature_names[response.indices[sorted_nzs]]
#results = [get_top_tf_idf_words(response,75) for response in responses]

In [ ]:
corpus['TandA'][0]

'NFTrig NFTrig application created educational tool teach trigonometry block chain Creation application includes front back development integration outside sources including MetaMask primary development languages include CSS JavaScript Solidity smart contract application hosted Moralis utilizing Web3 technical report describes application application smart contract design security considerations NFTrig application underwent significant testing validation prior Future suggestions recommendations fields education'

In [ ]:
results = [get_top_tf_idf_words(response,200) for response in responses]

In [ ]:
results[:1]

[array(['nftrig application', 'smart contract', 'application application',
        'nftrig nftrig application', 'report describes application',
        'recommendations fields education', 'recommendations fields',
        'prior future suggestions', 'prior future',
        'primary development languages', 'primary development',
        'outside sources including', 'outside sources', 'nftrig nftrig',
        'security considerations nftrig', 'nftrig application underwent',
        'nftrig application created', 'moralis utilizing web3',
        'moralis utilizing', 'metamask primary development',
        'metamask primary', 'languages include css', 'languages include',
        'application application smart', 'security considerations',
        'significant testing', 'integration outside sources',
        'significant testing validation', 'web3 technical',
        'validation prior future', 'validation prior',
        'utilizing web3 technical', 'utilizing web3',
        'underwent signif

In [ ]:
res = []
for doc_result in results:
    #monograms = [i for i in doc_result if len(i.split(' '))==1]
    bigrams = [i for i in doc_result if len(i.split(' '))==2]
    trigrams = [i for i in doc_result if len(i.split(' '))==3]
    delete1 = [e for e in bigrams for i in trigrams if e in i]

    #delete2 = [e for e in monograms for i in bigrams if e in i]

    #delete = delete1 + delete2
    res.append([item for item in doc_result if item not in delete1])

In [ ]:
corpus['keywords'] = res

In [3]:
dataset  = pd.read_excel('/content/Train.xlsx')

In [5]:
dataset.head(10)

,Phrases,comp_word
0,Machine Learning,1
1,Cybersecurity,1
2,Artificial Intelligence,1
3,Game Engineering,1
4,Design,1
5,Theory of Computation,1
6,Algorithms & Complexity Theory,1
7,Human-Computer Interaction,1
8,Computer Architecture,1
9,Scientific Computation,1


In [ ]:
# Import libraries and classes required
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [ ]:
X = dataset['Phrases']
y = dataset['comp_word']
# train test split (66% train - 33% test)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)
print('Training Data :', X_train.shape)
print('Testing Data : ', X_test.shape)

Training Data : (93,)
Testing Data :  (46,)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
X_train_cv.shape

(93, 150)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)

# transform X_test using CV
X_test_cv = cv.transform(X_test)

# generate predictions
predictions = lr.predict(X_test_cv)

In [ ]:
predictions

array([0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0])

In [ ]:
# confusion matrix
from sklearn import metrics
df = pd.DataFrame(metrics.confusion_matrix(y_test,predictions), index=[0,1], columns=['No','Yes'])
df

,No,Yes
0,18,4
1,4,20


In [ ]:
ter = ['neural network', 'pattern recognition', 'application takes', 'developers created','approaches control', 'significantly simpler', 'provides faithful']
X_test_cv = cv.transform(ter)
lr.predict(X_test_cv)

array([1, 1, 0, 0, 0, 0, 0])

In [ ]:
def getBetterKeywords(words):
    if len(words) == 0:
        return []
    else:
        transformed = cv.transform(words)
        predictions = lr.predict(transformed)
        ind = np.where(np.array(predictions) == 1)[0]
        keywords = np.take(words, ind)
        return keywords

In [ ]:
corpus['Keywords_new'] = corpus['keywords'].apply(getBetterKeywords)

In [ ]:
def limitWords(words):
  return words[:5]

In [ ]:
corpus['DocKeywords'] = corpus['Keywords_new'].apply(limitWords)

In [ ]:
corpus.DocKeywords

0                                   [block chain creation]
1        [perform local tasks, local tasks items, local...
2        [introduces research organizing, interactions ...
3        [experimental mutation data, outperforms model...
4        [explicit reward phenomenon, motivated behavio...
                               ...                        
19995    [neural machine translator, permit backpropaga...
19996    [time constant network, liquid time constant, ...
19997    [large rapidly growing, malicious benign promi...
19998    [solving implicit complementarity, smoothing i...
19999    [resonant coupling transient, perovskite phono...
Name: DocKeywords, Length: 20000, dtype: object

In [ ]:
corpus.columns

Index(['DocId', 'Title', 'Date', 'Authors', 'Abstract', 'Embeddings', 'Year',
       'Month', 'Link', 'TandA', 'cluster_id', 'Keywords_new', 'DocKeywords'],
      dtype='object')

In [ ]:
corpus.to_json('Archive.json')


# Get keywords with counts

In [ ]:
def flatten(x):
  merged = list(itertools.chain.from_iterable(x))
  return merged

In [ ]:
import itertools

In [ ]:
import collections

In [ ]:
keywords = pd.DataFrame()
for i in range(0,200):
  row = {}
  row['cluster_id'] = i
  cur = corpus[corpus['cluster_id']==i].Keywords_new
  crs = flatten(cur)
  #print(collections.Counter(crs))
  dicti = dict(collections.Counter(crs).most_common(30))
  #row['keywords'] = ','.join(list(dicti.keys()))
  row['keywords'] = dicti
  print(row)
  keywords = keywords.append(row, ignore_index = True)

{'cluster_id': 0, 'keywords': {'case arbitrary': 3, 'based sampling': 3, 'bound case': 3, 'concentration bounds': 2, 'based sampling replacement': 2, 'sampling replacement': 2, 'large class': 2, 'processes based': 2, 'prove concentration': 2, 'improve previous': 2, 'extending previous': 2, 'large number': 2, 'latent gaussian': 1, 'stationary vector': 1, 'count time': 1, 'gaussian series': 1, 'vector gaussian': 1, 'observed count': 1, 'marginally vector gaussian': 1, 'model akin vector': 1, 'model estimated relating': 1, 'models defined deterministic': 1, 'akin vector': 1, 'observed count series': 1, 'parameters estimated sparsely': 1, 'parameters marginally': 1, 'parameters marginally vector': 1, 'parametric model akin': 1, 'marginally vector': 1, 'marginal distribution counts': 1}}


<ipython-input-48-82615df56ab6>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keywords = keywords.append(row, ignore_index = True)


*After labelling:*

In [ ]:
keywords

,cluster_id,keywords,cluster_labels
0,0,"{'case arbitrary': 3, 'based sampling': 3, 'bo...",Sampling
1,1,"{'interactions generate': 2, 'effective lagran...",Non-linear Interactions and Gauge Theory
2,2,"{'machine learning': 405, 'learning models': 6...",Machine Learning and Feature Extraction
3,3,"{'propose novel': 9, 'propose novel framework'...",Reverse Engineering
4,4,"{'neural network': 25, 'trained model': 5, 'ba...",Bayesian Neural Network
...,...,...,...
195,195,"{'mean square': 15, 'square error': 14, 'mean ...",Models and Square errors
196,196,"{'graph neural': 109, 'neural networks': 79, '...",Graph Neural Networks and Deep Learning
197,197,"{'lattice qcd': 25, 'qcd calculations': 6, 'ex...",Gaussian methods and distributions
198,198,"{'upper triangular': 11, 'triangular matrices'...",Algebra and Matrices
